In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import models, layers
from tqdm import tqdm
from PIL import Image
import os
import matplotlib.pyplot as plt

In [ ]:
os.system("rm -rf ./*")

In [ ]:
!pip install -U tensorflow-addons
import tensorflow_addons as tfa
tqdm_callback = tfa.callbacks.TQDMProgressBar()

In [ ]:
raw_data = pd.read_csv("../input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv")
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [ ]:
hot_emotions = pd.get_dummies(raw_data['emotion'])
clear_hot_emotions = pd.DataFrame({"hot_emotions":[]})
for inx in tqdm(range(hot_emotions.shape[0])):
    hot_em = [list(hot_emotions.loc[inx])]
    tmp = pd.DataFrame({'hot_emotions':hot_em})
    clear_hot_emotions = clear_hot_emotions.append(tmp, ignore_index=True)

In [ ]:
raw_data = pd.concat([raw_data, clear_hot_emotions], axis=1).drop('emotion', axis=1)

In [ ]:
pixels = raw_data[" pixels"].copy()
three_dim_photos = []
for inx, el in tqdm(enumerate(pixels)):
  image = np.fromstring(el, dtype=int, sep=' ')
  three_image = [image, image, image]
  three_dim_photos.append(three_image)

In [ ]:
real_data = raw_data[[' Usage', 'hot_emotions']].copy()
real_data['three_dim_photos'] = three_dim_photos

In [ ]:
def parse_data(data):
  dt = np.zeros(shape=(len(data), 48, 48, 3))
  lb = np.array(list(map(list, data['hot_emotions'])))

  for inx, el in tqdm(enumerate(data['three_dim_photos'])):
    image = np.reshape(el, (48, 48, 3))
    dt[inx] = image


  return dt, lb

train_data, train_lab = parse_data(real_data[real_data[" Usage"] == "Training"])
test_data, test_lab = parse_data(real_data[real_data[" Usage"] == "PublicTest"])
val_data, val_lab = parse_data(real_data[real_data[" Usage"] == "PrivateTest"])

In [ ]:
train_data = train_data.astype(np.uint8)
test_data = test_data.astype(np.uint8)
val_data = val_data.astype(np.uint8)

In [ ]:
os.system("rm -rf ./*")

In [ ]:
os.system('mkdir -p /kaggle/working/imgs/train /kaggle/working/imgs/test /kaggle/working/imgs/val')

for inx, px_map in tqdm(enumerate(np.array(train_data[:]))):
    px_map = np.reshape(px_map, (48, 48, 3))
    image = Image.fromarray(px_map)
    image = image.convert('RGB')
    image.save('/kaggle/working/imgs/train/'+str(inx)+'.png')

for inx, px_map in tqdm(enumerate(np.array(test_data[:]))):
    px_map = np.reshape(px_map, (48, 48, 3))
    image = Image.fromarray(px_map)
    image = image.convert('RGB')
    image.save('/kaggle/working/imgs/test/'+str(inx)+'.png')

for inx, px_map in tqdm(enumerate(np.array(val_data[:]))):
    px_map = np.reshape(px_map, (48, 48, 3))
    image = Image.fromarray(px_map)
    image = image.convert('RGB')
    image.save('/kaggle/working/imgs/val/'+str(inx)+'.png')

In [ ]:
del train_data
del train_lab
del test_data
del test_lab
del val_data
del val_lab

In [ ]:
del hot_emotions
del clear_hot_emotions

In [ ]:
photos_dirs = ["./imgs/val", "./imgs/train", "./imgs/test"]

In [ ]:
scaled_imgs = pd.DataFrame({"converted_pixels":[]})

In [ ]:
def change_size(file_name, scale_to) -> np.ndarray:
    scale_factor = lambda dim, s : int(dim * s / 100)
    image = Image.open(file_name)
    wid, hei = image.size
    new_width = scale_factor(wid, scale_to)
    new_height = scale_factor(hei, scale_to)
    new_dim = (new_width, new_height)
    return image.resize(new_dim)

In [ ]:
train_max = 700
test_max = 300
val_max = 300

train_current = 0
test_current = 0
val_current = 0

for dirpath, dnames, fnames in os.walk("./imgs/train"):
    for file in tqdm(fnames):
        if train_current == train_max:
            break
        res = change_size("./imgs/train/" + file, 600)
        res_pixels = [list(np.array(res).flatten())]
        tmp = pd.DataFrame({"converted_pixels": res_pixels})
        scaled_imgs = scaled_imgs.append(tmp, ignore_index=True)
        del res
        del res_pixels
        train_current += 1
        
for dirpath, dnames, fnames in os.walk("./imgs/test"):
    for file in tqdm(fnames):
        if test_current == test_max:
            break
        res = change_size("./imgs/test/" + file, 600)
        res_pixels = [list(np.array(res).flatten())]
        tmp = pd.DataFrame({"converted_pixels": res_pixels})
        scaled_imgs = scaled_imgs.append(tmp, ignore_index=True)
        del res
        del res_pixels
        test_current += 1

        
for dirpath, dnames, fnames in os.walk("./imgs/val"):
    for file in tqdm(fnames):
        if val_current == val_max:
            break
        res = change_size("./imgs/val/" + file, 600)
        res_pixels = [list(np.array(res).flatten())]
        tmp = pd.DataFrame({"converted_pixels": res_pixels})
        scaled_imgs = scaled_imgs.append(tmp, ignore_index=True)
        del res
        del res_pixels
        val_current += 1

In [ ]:
scaled_imgs['converted_pixels']

# FREAKING HOORAY

In [ ]:
raw_data.iloc[:1300, :]

In [ ]:
scaled_imgs

In [ ]:
out_data = pd.DataFrame()

In [ ]:
out_data = pd.concat([raw_data.iloc[:1300, :], scaled_imgs], axis=1).drop(" pixels", axis=1)

In [ ]:
out_data

In [ ]:
out_data.to_csv("out_data.csv")